# Exploiter le guide des recettes

## Set variables

In [1]:
import mlflow
import dspy

# Enable autologging with all features
mlflow.dspy.autolog(
    log_compiles=True,    # Track optimization process
    log_evals=True,       # Track evaluation results
    log_traces_from_compile=True  # Track program traces during optimization
)

# Configure MLflow tracking
mlflow.set_tracking_uri("http://localhost:5000")  # Use local MLflow server
mlflow.set_experiment("06-workflow")

from dotenv import dotenv_values
config = dotenv_values("../../.env")

llm_model = config.get('ONLINE_LLM_MODEL')
api_key = config.get('ONLINE_LLM_API_KEY')

2026/01/05 21:19:30 INFO mlflow.tracking.fluent: Experiment with name '06-workflow' does not exist. Creating a new experiment.


## Configuration du llm sur dspy

In [2]:
import dspy

lm = dspy.LM(llm_model, api_key=api_key)
# Uncomment for local api call
#lm = dspy.LM(llm_model, api_base=api_base, track_usage=True, temperature=1.5, max_tokens=1024)

dspy.configure_cache(
    enable_disk_cache=False,
    enable_memory_cache=False,
)
dspy.configure(lm=lm)

## Créer une signature

In [3]:
import sys
from pathlib import Path
sys.path.append(str(Path("../../utils").resolve()))
from enum import Enum    
from file_reader import read_file


class EMAIL_INTENT(str, Enum):
    INFORMATION = "information"
    QUESTION = "question"
    ACTION = "action"
    COMPLAINT = "complaint"


class EMAIL_TONE(str, Enum):
    NEUTRAL = "neutral"
    POSITIVE = "positive"
    NEGATIVE = "negative"


class ClassifyEmail(dspy.Signature):
    """
    Analyze an incoming email to determine:
    - the sender's intent (what they expect)
    - the emotional tone (how they express it)

    The classification must be based on the email content only.
    """

    email: str = dspy.InputField(
        desc=(
            "Raw email content, including subject and body. "
            "May contain greetings, polite formulas, or emotional expressions."
        )
    )

    intent: EMAIL_INTENT = dspy.OutputField(
        desc=(
            "Primary intent of the email.\n"
            "- information: no reply or action required\n"
            "- question: asking for information\n"
            "- action: asking something to be done\n"
            "- complaint: expressing dissatisfaction or frustration"
        )
    )

    tone: EMAIL_TONE = dspy.OutputField(
        desc=(
            "Overall emotional tone of the email.\n"
            "Determine tone based on wording, urgency, politeness, "
            "and emotional expressions."
        )
    )


class SummarizeEmail(dspy.Signature):
    """
    Produce a short, factual summary of an informational email.
    The summary must capture the key message in one or two sentences.
    """

    email: str = dspy.InputField(
        desc="Informational email content to summarize."
    )

    summary: str = dspy.OutputField(
        desc="Concise summary capturing the essential information."
    )


class GenerateReply(dspy.Signature):
    """
    Generate a clear, professional reply to an email
    that asks a question or requests an action.
    """

    email: str = dspy.InputField(
        desc="Email requiring a factual or operational response."
    )

    reply: str = dspy.OutputField(
        desc=(
            "Polite, professional reply that addresses the request clearly. "
            "Do not include unnecessary empathy unless needed."
        )
    )


class EmpatheticReply(dspy.Signature):
    """
    Generate an empathetic response to a complaint email.

    The reply should:
    - acknowledge the issue
    - recognize the frustration
    - remain calm and professional
    - indicate next steps if possible
    """

    email: str = dspy.InputField(
        desc="Complaint email expressing dissatisfaction or frustration."
    )

    reply: str = dspy.OutputField(
        desc=(
            "Empathetic and professional response acknowledging the issue "
            "and reassuring the sender."
        )
    )


class EmailAgent(dspy.Module):

    def __init__(self):
        self.classifier = dspy.Predict(ClassifyEmail)
        self.summarizer = dspy.Predict(SummarizeEmail)
        self.replier = dspy.Predict(GenerateReply)
        self.empathetic = dspy.Predict(EmpatheticReply)

    def forward(self, email_file_path: str):
        email_content = read_file(email_file_path)
        analysis = self.classifier(email=email_content)
        result=None
        
        match analysis.intent:
            case EMAIL_INTENT.INFORMATION:
                result = self.summarizer(email=email_content).summary
            case EMAIL_INTENT.QUESTION:
                result = self.replier(email=email_content).reply
            case EMAIL_INTENT.COMPLAINT:
                result = self.empathetic(email=email_content).reply
            case _:
                raise Exception(f"Case for intent {analysis.intent} not implemented yet")
        
        return dspy.Prediction(
            analysis=analysis,
            result=result
        )

## Créer une prédiction

In [4]:
assistant = EmailAgent()

## Exécuter la prédiction

In [5]:
from rich import print


def process_email(email_file_path: str):
    response = assistant(email_file_path=email_file_path)

    print("----------")
    print(response)
    print("----------")
    # print(lm.history)
    # print("----------")
    # dspy.inspect_history()
    # print("----------")


process_email("../../assets/emails/001-mail.md")
process_email("../../assets/emails/002-mail.md")
process_email("../../assets/emails/003-mail.md")

----------

Prediction(
    analysis=Prediction(
    intent=<EMAIL_INTENT.INFORMATION: 'information'>,
    tone=<EMAIL_TONE.NEUTRAL: 'neutral'>
),
    result='The quarterly meeting is scheduled for next Tuesday at 2 PM in room Atlas, with the agenda to be sent 
the following day.'
)

----------

----------

Prediction(
    analysis=Prediction(
    intent=<EMAIL_INTENT.QUESTION: 'question'>,
    tone=<EMAIL_TONE.NEUTRAL: 'neutral'>
),
    result='Bonjour M. Paul,\n\nOui, le VPN fonctionne également depuis l’étranger. Si vous rencontrez des 
difficultés pour y accéder, n’hésitez pas à nous contacter pour assistance.\n\nCordialement,\n[Votre Nom]\n[Votre 
Position]\n[Votre Contact]'
)

----------

Exception: Case for intent EMAIL_INTENT.ACTION not implemented yet

[Trace(trace_id=tr-c7eca71c4ff60ed757900d2c652ab8b3), Trace(trace_id=tr-efc6f3a5bf5092ebec557c207dfc2d02), Trace(trace_id=tr-24ffb6c1555847efdca946335963185c)]